In [1]:
# Import dependencies
from citipy import citipy
import pandas as pd
import numpy as np
import json 
import requests
import timeit

from datetime import datetime
from config import weather_api_key

In [2]:
# Generate a new set of 1,500 random latitudes and longitudes.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lats_lngs = zip(lats, lngs)
lats_lngs

In [3]:
# Add the latitudes and longitudes in a list.
coordinates = list(lats_lngs)
print (coordinates)

[(71.27799302792823, 108.66648569872297), (79.20046602188896, -162.30808021765537), (76.0133078182686, -75.48682007716666), (28.108951546641336, 72.1462321310749), (-69.70143955678286, 19.056175346245624), (80.91730506708103, -137.652275811393), (-48.95542875718293, -122.36396408858118), (-47.623033274468035, -3.1833950488548908), (25.629132359096005, 42.360634270391586), (65.70820249299089, 46.95481989728364), (58.88167250538257, 58.8254022609126), (-57.970172631460784, 128.21175981003285), (-54.664690670663624, -43.10749275829349), (82.11273641736065, 151.19656822083203), (-7.910486870478181, 120.69061487199036), (21.078930151175214, 42.28165849547656), (51.88718718317554, -142.9553688058096), (7.794905372733467, 33.453586764101004), (30.93565018838646, 105.51770955009897), (24.745268689794955, 51.812080816588235), (-51.12577320287165, 134.36460547890522), (65.59953583399414, 129.15638840482956), (-61.28107387163526, 50.73265492619598), (44.39996175166567, 165.89037915114955), (-81.8

In [4]:
# import the citipy module.
from citipy import citipy

In [5]:
#Create a list for holding the names of cities.
cities = []

#Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
#If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
        
# Print the city count to confirm sufficient count.
len(cities)

606

In [6]:
# Import the requests library.
import requests

#Import the API key.
from config import weather_api_key

In [7]:
# Starting URL for weather map API call.
url="http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [8]:
#Import the datetime module from the datetime library.
from datetime import datetime

In [9]:
#Create an empty list to hold the weather data.
city_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

#Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in our list.
for i, city in enumerate(cities):
    
    # Group cities in sets of 50 for logging purposes.
    if(i % 50 ==0 and i>=50):
        set_count += 1
        record_count = 1
        
    #Create endpoint url for each city.
    city_url = url + "&q=" + city
        
    #log the url,record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
        
    #Add 1 to the record count.
    record_count += 1
        
    #Run an API request for each of the cities.
    try:
         #Parse the JSON and retrieve the data.
        city_weather = requests.get(city_url).json()
        #Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_description=city_weather["weather"][0]["description"]
            
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        #Rain data
        try:
            city_rain = city_weather["rain"]["3h"]
            
        except:
            city_rain = 0.00 
            
        try:
            city_snow = city_weather["snow"]["3h"]
        except:
            city_snow = 0.00
            
            
        #Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Date": city_date,
                            "Lat": city_lat,
                             "Lng": city_lng,
                             "Max Temp": city_max_temp,
                             "Humidity": city_humidity,
                             "Cloudiness": city_clouds,
                             "Wind Speed": city_wind,                                                         
                             "Current Description":city_weather_description})
        
    #If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass
    
#Indicate the data loading is complete.
print("------------------------------")
print("Data Retrieval Complete       ")
print("------------------------------")          

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | saskylakh
Processing Record 2 of Set 1 | barrow
Processing Record 3 of Set 1 | qaanaaq
Processing Record 4 of Set 1 | phalodi
Processing Record 5 of Set 1 | bredasdorp
Processing Record 6 of Set 1 | tuktoyaktuk
Processing Record 7 of Set 1 | rikitea
Processing Record 8 of Set 1 | cape town
Processing Record 9 of Set 1 | buraydah
Processing Record 10 of Set 1 | leshukonskoye
Processing Record 11 of Set 1 | medvedka
Processing Record 12 of Set 1 | mount gambier
Processing Record 13 of Set 1 | mar del plata
Processing Record 14 of Set 1 | chokurdakh
Processing Record 15 of Set 1 | ruteng
Processing Record 16 of Set 1 | mecca
Processing Record 17 of Set 1 | sitka
Processing Record 18 of Set 1 | gambela
Processing Record 19 of Set 1 | taihe
Processing Record 20 of Set 1 | doha
Processing Record 21 of Set 1 | portland
Processing Record 22 of Set 1 | sangar
Processing Record 23 of Set 1 | east london
Pr

Processing Record 37 of Set 4 | olinda
Processing Record 38 of Set 4 | nome
Processing Record 39 of Set 4 | cairns
Processing Record 40 of Set 4 | nova olinda do norte
Processing Record 41 of Set 4 | acapulco
Processing Record 42 of Set 4 | pangnirtung
Processing Record 43 of Set 4 | sur
Processing Record 44 of Set 4 | lata
Processing Record 45 of Set 4 | himora
City not found. Skipping...
Processing Record 46 of Set 4 | longyearbyen
Processing Record 47 of Set 4 | makakilo city
Processing Record 48 of Set 4 | barabai
Processing Record 49 of Set 4 | faro
Processing Record 50 of Set 4 | korla
Processing Record 1 of Set 5 | jiuquan
Processing Record 2 of Set 5 | oksovskiy
Processing Record 3 of Set 5 | khatanga
Processing Record 4 of Set 5 | ribeira grande
Processing Record 5 of Set 5 | magadan
Processing Record 6 of Set 5 | sayyan
Processing Record 7 of Set 5 | krasnoselkup
Processing Record 8 of Set 5 | san vicente
Processing Record 9 of Set 5 | nouadhibou
Processing Record 10 of Set 5

Processing Record 30 of Set 8 | semey
Processing Record 31 of Set 8 | thinadhoo
Processing Record 32 of Set 8 | tanda
Processing Record 33 of Set 8 | berlevag
Processing Record 34 of Set 8 | chuy
Processing Record 35 of Set 8 | gamba
Processing Record 36 of Set 8 | male
Processing Record 37 of Set 8 | sao filipe
Processing Record 38 of Set 8 | raudeberg
Processing Record 39 of Set 8 | mnogovershinnyy
Processing Record 40 of Set 8 | jamsa
Processing Record 41 of Set 8 | nyaunglebin
Processing Record 42 of Set 8 | egvekinot
Processing Record 43 of Set 8 | guerrero negro
Processing Record 44 of Set 8 | at-bashi
Processing Record 45 of Set 8 | santa rosa
Processing Record 46 of Set 8 | lebu
Processing Record 47 of Set 8 | port hedland
Processing Record 48 of Set 8 | kaeo
Processing Record 49 of Set 8 | yambio
Processing Record 50 of Set 8 | les cayes
Processing Record 1 of Set 9 | marsh harbour
Processing Record 2 of Set 9 | perelyub
Processing Record 3 of Set 9 | jacmel
Processing Record 

Processing Record 20 of Set 12 | boromo
Processing Record 21 of Set 12 | phalaborwa
Processing Record 22 of Set 12 | san carlos de bariloche
Processing Record 23 of Set 12 | cidreira
Processing Record 24 of Set 12 | fort frances
Processing Record 25 of Set 12 | itarema
Processing Record 26 of Set 12 | montabaur
Processing Record 27 of Set 12 | batemans bay
Processing Record 28 of Set 12 | calabozo
Processing Record 29 of Set 12 | talnakh
Processing Record 30 of Set 12 | obo
Processing Record 31 of Set 12 | orsk
Processing Record 32 of Set 12 | tabiauea
City not found. Skipping...
Processing Record 33 of Set 12 | ekhabi
Processing Record 34 of Set 12 | grand centre
City not found. Skipping...
Processing Record 35 of Set 12 | rehoboth
Processing Record 36 of Set 12 | itaqui
Processing Record 37 of Set 12 | palabuhanratu
City not found. Skipping...
Processing Record 38 of Set 12 | orange cove
Processing Record 39 of Set 12 | yashan
Processing Record 40 of Set 12 | barcelona
Processing Rec

In [11]:
len(city_data)

562

In [14]:
# Create a city dataframe.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Saskylakh,RU,2022-08-01 01:03:26,71.9167,114.0833,45.64,77,100,10.36,overcast clouds
1,Barrow,US,2022-08-01 01:03:26,71.2906,-156.7887,35.62,80,100,9.22,overcast clouds
2,Qaanaaq,GL,2022-08-01 01:03:27,77.4840,-69.3632,44.74,64,100,2.48,overcast clouds
3,Phalodi,IN,2022-08-01 01:03:27,27.1333,72.3667,81.90,80,81,17.34,broken clouds
4,Bredasdorp,ZA,2022-08-01 01:03:27,-34.5322,20.0403,41.77,83,0,3.56,clear sky
5,Tuktoyaktuk,CA,2022-08-01 01:03:27,69.4541,-133.0374,50.31,76,97,12.50,overcast clouds
6,Rikitea,PF,2022-08-01 01:03:28,-23.1203,-134.9692,72.46,73,100,23.82,light rain
7,Cape Town,ZA,2022-08-01 01:03:28,-33.9258,18.4232,55.38,75,0,8.99,clear sky
8,Buraydah,SA,2022-08-01 01:03:28,26.3260,43.9750,84.18,51,98,12.82,overcast clouds
9,Leshukonskoye,RU,2022-08-01 00:58:36,64.8989,45.7622,57.78,97,100,2.28,overcast clouds


In [13]:
#Export the dataframe to csv file.
city_data_df.to_csv("Weather_Database/WeatherPy_database.csv", index_label="City ID")